In [1]:
with open("input.txt") as f:
    lines = list(map(str.rstrip, f))

In [140]:
import re
from collections import namedtuple, defaultdict


Subj = namedtuple("Subj", ("adverb", "colour"))
Obj = namedtuple("Obj", ("quantity", "adverb", "colour"))


def parse_rule(r):
    subj, *objs = re.findall(r"(?P<subj_clause>(?:\d+ )?\w+ \w+) bags?", r)
    
    try:
        return Subj(*subj.split()), list(Obj(*obj.split()) for obj in objs)
    except:
        if "no other bags" in r:
            return Subj(*subj.split()), list()
        else:
            raise ValueError(f"{r} - {subj}, {objs}")

def build_grammar(rs):
    grammar = {
        "contained_by": defaultdict(set)
    }
    
    for r in rs:
        subj, objs = parse_rule(r)
        k_s = (subj.adverb, subj.colour)
        
        for o in objs:
            k_o = (o.adverb, o.colour)
            grammar["contained_by"][k_o].add(k_s)
    
    return grammar

In [141]:
g = build_grammar(lines)

In [142]:
def search_can_contain(adverb, colour):
    visited_nodes = set()
    
    def dfs(node):
        if node not in visited_nodes:
            visited_nodes.add(node)
            
            for neighbour in g["contained_by"][node]:
                dfs(neighbour)
    
    dfs((adverb, colour))
    
    return len(visited_nodes) - 1


In [143]:
search_can_contain("shiny", "gold")

268

In [146]:
def build_grammar_2(rs):
    grammar = {
        "contains": defaultdict(set),
    }
    
    for r in rs:
        subj, objs = parse_rule(r)
        k_s = (subj.adverb, subj.colour)
        
        for o in objs:
            k_o = (o.quantity, o.adverb, o.colour)
            grammar["contains"][k_s].add(k_o)
    
    return grammar


g_2 = build_grammar_2(lines)


def count_must_contain(adverb, colour, debug=True):
    TAB = "\t"
    
    def dfs(adverb, colour, quantity=1, depth=0):  # not _really_ a DFS but ok because it's acyclic - otherwise key would have to be entire path
        node = (adverb, colour)
                    
        if g_2["contains"][node]:
            quantity = int(quantity)
            
            ret = quantity + (quantity * sum(
                dfs(
                    adverb=a, colour=c, quantity=q, depth=depth+1
                )
                for q, a, c in g_2["contains"][node]
            ))
            if debug:
                print(f"{TAB * depth} {quantity} {adverb} {colour} - returns {ret} as product+leaf")
            return ret
        else:
            ret = int(quantity)
            if debug:
                print(f"{TAB * depth} {quantity} {adverb} {colour} - returns {ret} as leaf")
            return ret
    
    return dfs(adverb, colour) - 1

In [147]:
count_must_contain("shiny", "gold")

		 3 clear red - returns 3 as leaf
			 5 muted white - returns 5 as leaf
			 4 dull lime - returns 4 as leaf
			 2 faded fuchsia - returns 2 as leaf
			 2 dim gold - returns 2 as leaf
		 3 dotted maroon - returns 42 as product+leaf
	 5 muted black - returns 230 as product+leaf
				 3 muted magenta - returns 3 as leaf
					 1 faded fuchsia - returns 1 as leaf
						 5 dark cyan - returns 5 as leaf
					 2 dark purple - returns 12 as product+leaf
					 4 clear red - returns 4 as leaf
				 2 dull red - returns 36 as product+leaf
			 4 mirrored plum - returns 160 as product+leaf
					 1 faded fuchsia - returns 1 as leaf
						 5 dark cyan - returns 5 as leaf
					 2 dark purple - returns 12 as product+leaf
					 4 clear red - returns 4 as leaf
				 5 dull red - returns 90 as product+leaf
					 4 clear red - returns 4 as leaf
				 5 dotted plum - returns 25 as product+leaf
					 4 dull lime - returns 4 as leaf
								 3 dark cyan - returns 3 as leaf
								 3 striped crimson - returns 3 a

7867